In [1]:
import recordlinkage
import pandas as pd
import numpy
from recordlinkage.base import BaseIndexAlgorithm
import time

In [82]:
final = pd.read_csv("./src/final_dataset.csv")

In [83]:
final = final.drop(columns=['Unnamed: 0'])

In [84]:
final.head()

,Unnamed: 0,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,0.0,LACEWORK,UNITED STATES,CYBERSECURITY,2015.0,NaN,NaN,NaN,NaN,NaN
1,1.0,TIPALTI,UNITED STATES,FINTECH,2010.0,NaN,NaN,NaN,NaN,NaN
2,2.0,TEMPUS,UNITED STATES,HEALTH,2015.0,NaN,NaN,NaN,NaN,NaN
3,3.0,ANDURIL,UNITED STATES,ARTIFICIAL INTELLIGENCE,2017.0,NaN,NaN,NaN,NaN,NaN
4,4.0,BOLT,ESTONIA,AUTO & TRANSPORTATION,2013.0,NaN,NaN,NaN,NaN,NaN


In [44]:
final["name"]

0                                           LACEWORK
1                                            TIPALTI
2                                             TEMPUS
3                                            ANDURIL
4                                               BOLT
                             ...                    
173029                      SOFTBANK ROBOTICS UK LTD
173030                               COMUNE DI VOLLA
173031    SUDDATH RELOCATION SYSTEMS OF ORLANDO, INC
173032                          MIDASPLAYER.COM LTD.
173033                                  DRILLMEC SPA
Name: name, Length: 173034, dtype: object

In [85]:
len(final)

173034

In [86]:
sorted_final = final.sort_values('name')

In [87]:
sorted_final = sorted_final.reset_index(drop=True)

In [88]:
sorted_final.head()

,Unnamed: 0,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,5720.0,\nADDUS HOMECARE,USA,NaN,NaN,1.690000e+09,NaN,NaN,NaN,NaN
1,148737.0,#SINOB,GERMANY,RETAIL,2015.0,NaN,4.0,14.0,HTTPS://WWW.SINOB.DE/,NaN
2,66863.0,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIN,SPAIN,NaN,NaN,8.700000e+08,NaN,NaN,NaN,NaN
3,43845.0,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN,SPAIN,NaN,NaN,8.200000e+08,NaN,NaN,NaN,NaN
4,111815.0,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN,SPAIN,NaN,NaN,8.200000e+08,NaN,NaN,NaN,NaN


In [49]:
sorted_final.loc[0]

Unnamed: 0              5720.0
name          \nADDUS HOMECARE
country                    USA
sector                     NaN
founded                    NaN
marketcap         1690000000.0
revenue                    NaN
employees                  NaN
links                      NaN
ceo                        NaN
Name: 0, dtype: object

In [89]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [90]:
similar("USA", "UNITED STATES")

0.375

In [91]:
class CustomIndex(BaseIndexAlgorithm):
    def _link_index(self, df_a, df_b):
        t0 = time.time()
        indici1=[]
        indici2=[]
        for i in range(0, len(final)):
            if(i%500 == 0):
                print(i)
            if(i<100):
                for j in range(0, (i+100)):
                    if(similar(df_a.loc[i, 'name'], df_a.loc[j, 'name'])>0.35):
                        indici1.append(i)
                        indici2.append(j)
                if(i<10):
                    print(df_a.loc[i,'name'])
            elif(i>172932):
                for j in range((i-100), len(final)):
                    if(similar(df_a.loc[i, 'name'], df_a.loc[j, 'name'])>0.35):
                        indici1.append(i)
                        indici2.append(j)
            else:
                for j in range((i-100), (i+100)):
                    if(similar(df_a.loc[i, 'name'], df_a.loc[j, 'name'])>0.35):
                        indici1.append(i)
                        indici2.append(j)
        
        indici = [indici1, indici2]
        t1 = time.time()
        print(t1-t0)
        return pd.MultiIndex.from_arrays(indici, names=('first', 'second'))                  

In [92]:
indexer = CustomIndex()
candidate_pairs = indexer.index(sorted_final, sorted_final)


0

ADDUS HOMECARE
#SINOB
(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIN
(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN
(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN
(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN
(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN
(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN
(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN
(HLBANK) HONG LEONG BANK
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
520

In [94]:
candidate_df = candidate_pairs.to_frame()

In [96]:
candidate_df.head()

0   1
0 0   0   0
  90  0  90
  93  0  93
1 1   1   1
2 2   2   2

In [100]:
candidate_pairs

MultiIndex([(     0,      0),
            (     0,     90),
            (     0,     93),
            (     1,      1),
            (     2,      2),
            (     2,      3),
            (     2,      4),
            (     2,      5),
            (     2,      6),
            (     2,      7),
            ...
            (173032, 172997),
            (173032, 172998),
            (173032, 172999),
            (173032, 173000),
            (173032, 173028),
            (173032, 173029),
            (173032, 173030),
            (173032, 173031),
            (173032, 173032),
            (173033, 173033)],
           length=18650932)

In [95]:
candidate_df.to_csv("./src/candidate_pairs.csv")

In [105]:
candidate_df = pd.read_csv("./src/candidate_pairs.csv")

In [106]:
candidate_df

,Unnamed: 0,Unnamed: 1,0,1
0,0,0,0,0
1,0,90,0,90
2,0,93,0,93
3,1,1,1,1
4,2,2,2,2
...,...,...,...,...
18650927,173032,173029,173032,173029
18650928,173032,173030,173032,173030
18650929,173032,173031,173032,173031
18650930,173032,173032,173032,173032


In [108]:
candidate_df = candidate_df.drop(columns=['0', '1'])

In [112]:
candidate_df = candidate_df.rename(columns={"Unnamed: 0": "r1", "Unnamed: 1": "r2"})

In [113]:
candidate_df.head()

,r1,r2
0,0,0
1,0,90
2,0,93
3,1,1
4,2,2


In [114]:
candidate_pairs = pd.MultiIndex.from_frame(candidate_df, names=['state', 'observation'])

In [115]:
candidate_pairs

MultiIndex([(     0,      0),
            (     0,     90),
            (     0,     93),
            (     1,      1),
            (     2,      2),
            (     2,      3),
            (     2,      4),
            (     2,      5),
            (     2,      6),
            (     2,      7),
            ...
            (173032, 172997),
            (173032, 172998),
            (173032, 172999),
            (173032, 173000),
            (173032, 173028),
            (173032, 173029),
            (173032, 173030),
            (173032, 173031),
            (173032, 173032),
            (173033, 173033)],
           names=['state', 'observation'], length=18650932)

In [116]:
compare = recordlinkage.Compare()

compare.string('name', 'name', method='jarowinkler', label='name')
compare.string('country', 'country', method='jarowinkler', label='country', missing_value = 0.1)
compare.string('sector', 'sector', label='sector', method='jarowinkler', missing_value = 0.1)
compare.exact('founded', 'founded', missing_value = 0.1, label = 'founded')
compare.exact('marketcap', 'marketcap', missing_value = 0.1, label = 'marketcap')
compare.exact('revenue', 'revenue', missing_value = 0.1, label = 'revenue')
compare.exact('employees', 'employees', missing_value = 0.1, label = 'employees')
compare.string('links', 'links', method='jarowinkler', label='links', missing_value = 0.1)
compare.string('ceo', 'ceo', method='jarowinkler', label='ceo', missing_value = 0.1)

<Compare>

In [117]:
training_features = compare.compute(candidate_pairs, sorted_final, sorted_final)

In [118]:
training_features['score'] = training_features.loc[:, 'name':'ceo'].sum(axis=1)

In [ ]:
training_matches = training_features[training_features.sum(axis=1) > 2].reset_index()

In [ ]:
training_matches.to_csv("./src/prova_matching.csv")

In [81]:
training_matches.head(1205572)

,Unnamed: 0,Unnamed: 1,name,country,sector,founded,marketcap,revenue,employees,links,ceo,score
0,0,90,0.548990,0.477778,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.726768
1,0,93,0.533333,0.500000,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.733333
2,1,1,1.000000,1.000000,1.0,1.0,0.1,1.0,1.0,1.0,0.1,7.200000
3,2,2,1.000000,1.000000,0.1,0.1,1.0,0.1,0.1,0.1,0.1,3.600000
4,2,3,0.995918,1.000000,0.1,0.1,0.0,0.1,0.1,0.1,0.1,2.595918
...,...,...,...,...,...,...,...,...,...,...,...,...
1205567,11238,11146,0.590909,0.470085,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.760995
1205568,11238,11147,0.590909,0.470085,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1.760995
1205569,11238,11155,0.674592,0.666667,0.1,0.1,0.1,0.1,0.1,0.1,0.1,2.041259
1205570,11238,11156,0.618687,1.000000,0.1,0.1,0.0,0.1,0.1,0.1,0.1,2.218687
